In [1]:
import pandas as pd

import sys
sys.path.append("src")

from merging_with_id import merging, create_id

In [2]:
# load datasets
datasets = {}
years = range(2018, 2021)
for year in years:
    print(f"Schools stats {year} Loading...")
    file_name = f'ZZSO-{year}.xlsx'
    dataset = pd.read_excel(f"./fin_data/{file_name}", header=1, dtype='str')
    datasets.update({year:dataset})
    print("success")

Schools stats 2018 Loading...
success
Schools stats 2019 Loading...
success
Schools stats 2020 Loading...
success


In [3]:
for year in years:
    print(year,'\n', list(datasets[year].columns))

2018 
 ['№ Області', 'Область', 'Тип територіальної одиниці', '№№', 'Назва тер. Одиниці', 'Найменування закладу', 'Ступінь закладу ( I, I-II, I-III)', 'Загальна площа приміщень школи ( без господарських будівель) (кв.м.)', 'Кількість штатних працівників педагогічного персоналу  разом', 'Кількість штатних працівників непедагогічного персоналу  разом', 'Кількість штатних працівників педагогічного персоналу  пенсійного віку', 'Питома вага непедагогічного персоналу у загальній кількості працівників, %**', 'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**', 'Кількість учнів разом', 'Кількість класів разом', 'Фактична наповнюваність класів, учнів на клас**', 'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**', 'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)', 'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (план)', 'Видатки на 1 учня, 2018, тис. 

In [4]:
rename_dict = {
    "Область": "regname",
    "Тип територіальної одиниці": "tertype",
    "Тип школи: міська / сільська": 'eotype',
    "Назва тер. Одиниці": "areaname",
    "Найменування закладу": "eoname",
    "Ступінь закладу ( I, I-II, I-III)": "eolevel",
    "Загальна площа приміщень школи ( без господарських будівель) (кв.м.)": "eosize",
    "Кількість штатних працівників педагогічного персоналу  разом": "teachstuff",
    "Кількість штатних працівників непедагогічного персоналу  разом": "nonteachstuff",
    "Кількість штатних працівників педагогічного персоналу  пенсійного віку": "teachstuffretage",
    "Кількість учнів разом": "pupils",
    "Кількість класів разом": "classes",
    "Разом кількість учнів, та дітей, які здобувають дошкільну освіту": "pupilswithpreschool",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)": "opex",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)": "opex",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (касові видатки)": "opex",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (план)": "opexplan",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)": "opexplan",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)": "opexplan",
    "Опорна": "hub",
    "Код бюджету": "budgetcode",
    "Форма власності": "ownership",
    "Код ДІСО": "DISO",
    "ЄДРПОУ": "EDRPOU"
    }

In [5]:
for year in years:
    datasets[year].rename(columns=rename_dict, inplace=True)
    to_drop = [col for col in datasets[year].columns if col not in rename_dict.values()]
    datasets[year].drop(columns = to_drop, inplace = True)
    datasets[year]['year'] = year


# Data cleaning 2020

## Location region

In [6]:
# load location dictionary
locations = pd.read_csv('./final_tables/locations.csv', encoding='utf-8', dtype=str)
locations

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797
...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,0026010000,OC26010000000000000
29215,Велика Британія,м.Абериствіт,м.Абериствіт,0008030000,OC08030000000000000
29216,США,м.Нью-Йорк,м.Нью-Йорк,0026020000,OC26020000000000000
29217,США,м.Х'юстон,м.Х'юстон,0026040000,OC26040000000000000


In [7]:
datasets[2020].regname = datasets[2020].regname.apply(lambda x: 'м.Київ' if x == 'місто Київ' or x == 'м.Київ' else x+' область')
datasets[2020].regname.unique()

array(['Вінницька область', 'Волинська область',
       'Дніпропетровська область', 'Донецька область',
       'Житомирська область', 'Закарпатська область',
       'Запорізька область', 'Івано-Франківська область',
       'Київська область', 'Кіровоградська область', 'Луганська область',
       'Львівська область', 'Миколаївська область', 'Одеська область',
       'Полтавська область', 'Рівненська область', 'Сумська область',
       'Тернопільська область', 'Харківська область',
       'Херсонська область', 'Хмельницька область', 'Черкаська область',
       'Чернівецька область', 'Чернігівська область', 'м.Київ'],
      dtype=object)

In [8]:
locations['KOATUU_2020_reg'] = locations['KOATUU_2020'].str[:2]
locations

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023,KOATUU_2020_reg
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454,23
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567,68
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918,12
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137,73
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797,48
...,...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,0026010000,OC26010000000000000,00
29215,Велика Британія,м.Абериствіт,м.Абериствіт,0008030000,OC08030000000000000,00
29216,США,м.Нью-Йорк,м.Нью-Йорк,0026020000,OC26020000000000000,00
29217,США,м.Х'юстон,м.Х'юстон,0026040000,OC26040000000000000,00


In [9]:
loc = locations[['regname', 'KOATUU_2020_reg']]
loc.drop_duplicates(inplace= True)
loc.reset_index(drop=True, inplace=True)
loc

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_8256/3378581834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc.drop_duplicates(inplace= True)


,regname,KOATUU_2020_reg
0,Запорізька область,23
1,Хмельницька область,68
2,Дніпропетровська область,12
3,Чернівецька область,73
4,Миколаївська область,48
5,Донецька область,14
6,Тернопільська область,61
7,Кіровоградська область,35
8,Одеська область,51
9,Львівська область,46


In [10]:
dataset_2020 = datasets[2020].merge(loc[['regname', 'KOATUU_2020_reg']], on=['regname'], how='left')
dataset_2020 .head()

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,9,4388.111,5006.502,NaN,02553000000,комунальна,7137,25844555,2020,05
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,12,6523.777,8614.601,NaN,02553000000,комунальна,6936,25844992,2020,05
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,21,9273.777,12627.197,NaN,02553000000,комунальна,7149,25844928,2020,05
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,19,8654.501,11029.117,NaN,02553000000,комунальна,7191,25844970,2020,05
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,31,14833.298,19559.764,NaN,02553000000,комунальна,6976,23101028,2020,05


In [11]:
set(dataset_2020.eolevel.unique())

{'0', 'I', 'I-II', 'I-III', 'I-IIІ', 'II', 'II-III', 'III', 'ІІ-ІІІ'}

In [12]:
values_to_drop = ['I', 'I-II', 'II']

dataset_2020  = dataset_2020[~dataset_2020.eolevel.isin(values_to_drop)]
dataset_2020.reset_index(drop=True, inplace=True)
dataset_2020 

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,9,4388.111,5006.502,NaN,02553000000,комунальна,7137,25844555,2020,05
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,12,6523.777,8614.601,NaN,02553000000,комунальна,6936,25844992,2020,05
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,21,9273.777,12627.197,NaN,02553000000,комунальна,7149,25844928,2020,05
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,19,8654.501,11029.117,NaN,02553000000,комунальна,7191,25844970,2020,05
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,31,14833.298,19559.764,NaN,02553000000,комунальна,6976,23101028,2020,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,30,19606.3,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,34,22065.4,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,18,15793.70295055802,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,30,20353.2,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80


In [13]:
# check whether there are duplicates by name and location
dataset_2020[dataset_2020.duplicated(subset=['eoname', 'KOATUU_2020_reg'], keep=False)]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
6473,Тернопільська область,село,Бучацька міська ТГ,Озерянська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,22,12,2,133,11,4379.4,5352.2,NaN,19555000000,комунальна,17857,24623237,2020,61
6610,Тернопільська область,село,Борщівська міська ТГ,Озерянська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,25,16,1,196,11,6062.3,7561.6,NaN,19527000000,комунальна,17807,24620894,2020,61


In [14]:
dataset_2020['EDRPOU_checked'] = ''

In [15]:
dataset_2020.loc[dataset_2020.EDRPOU == '24623237', 'EDRPOU_checked'] = '24623237'
dataset_2020.loc[dataset_2020.EDRPOU == '24620894', 'EDRPOU_checked'] = '24620894'

In [16]:
dataset_2020[dataset_2020.EDRPOU.str.strip().str.len() != 8].EDRPOU.unique()

array(['2141288', '2141294', '2141302', '2141377', '62059', '7995',
       '2141383', '0', '2141408', '11', '2141354', '2141650', '4334672',
       '2142201', '2124769', '2133320', '2360228', '6669359', '6669419',
       '6669879', '6669750', '6669833', '6669856', '6669773', '6669827',
       '6670121', '6669744', '6669738', '6669786', '6670664', '6670635',
       '6670606', '6670753', '6670919', '6671155', '6671445', '6671267',
       '6671149', '6671497', '6671422', '6671468', '6671110', '6671474',
       '6671178', '6670374', '6670428', '6670405', '6670411', '6670380',
       '6670397', '6670368', '6670474', '6669263', '6669313', '6669655',
       '6669721', '6669394', '6669454', '6669253', '6669247', '6669336',
       '6671451', '6671439', '6671161', '6669804', '6669862', '6669649',
       '6669810', '6669603', '6669796', '6669840', '6670701', '6670776',
       '6670687', '6670693', '6670718', '6670552', '6670581', '6670598',
       '6670730', '6670782', '6670569', '6670575', '6670

In [17]:
dataset_2020.EDRPOU = dataset_2020.EDRPOU.apply(lambda x: x.zfill(8) if isinstance(x, str) and len(x) >= 6 else None)

In [18]:
dataset_2020[(dataset_2020.EDRPOU.isna())].eoname.unique()

array(['Турбівська загальноосвітня школа І-ІІІ ст. №2 _x000D_\nЛиповецької районної ради _x000D_\nВінницької області_x000D_\n_x000D_\n',
       'Середня загальноосвітня школа І - ІІІ ступенів села Уладівка Літинський район Вінницька область',
       'Середня загальноосвітня школа І-ІІІ ступенів с. Івча',
       'Комунальний заклад "Загальноосвітня школа І-ІІІ ступенів № 19 Вінницької міської ради"',
       'Старобільська загальноосвітня школа ІІ-ІІІ ступенів №4 Старобільської районної ради Луганської області'],
      dtype=object)

In [19]:
set_edrpou = {
    'Турбівська загальноосвітня школа І-ІІІ ст. №2 _x000D_\nЛиповецької районної ради _x000D_\nВінницької області_x000D_\n_x000D_\n': '21723820',
    'Середня загальноосвітня школа І - ІІІ ступенів села Уладівка Літинський район Вінницька область':'26229088',
    'Середня загальноосвітня школа І-ІІІ ступенів с. Івча':'26223660',
    'Комунальний заклад "Загальноосвітня школа І-ІІІ ступенів № 19 Вінницької міської ради"':'22103539',
    'Старобільська загальноосвітня школа ІІ-ІІІ ступенів №4 Старобільської районної ради Луганської області':'33199525'}

In [20]:
for school in set_edrpou:
    dataset_2020.loc[dataset_2020['eoname'] == school, 'EDRPOU'] = set_edrpou[school]
    dataset_2020.loc[dataset_2020['eoname'] == school, 'EDRPOU_checked'] = set_edrpou[school]

In [21]:
create_id(dataset_2020, 'eoname', 'id')

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,5006.502,NaN,02553000000,комунальна,7137,25844555,2020,05,,загальноосвтнйнавчальновиховнийкомплекс3ступшк...
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,8614.601,NaN,02553000000,комунальна,6936,25844992,2020,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,12627.197,NaN,02553000000,комунальна,7149,25844928,2020,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,11029.117,NaN,02553000000,комунальна,7191,25844970,2020,05,,загальноосвтнясередняшкола3ступ5козятинськавни...
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,19559.764,NaN,02553000000,комунальна,6976,23101028,2020,05,,комунальнийнавчальнийзакладспецалзованашкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80,,гмназя237
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80,,середнязагальноосвтняшкола3ступ195менвкудряшов...
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80,,спецалзованашкола3ступ73зпоглибленимвивченямук...
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80,,спецалзованашкола131мкив


## MON 2023

In [22]:
#load data
schools_data = pd.read_excel('./school_data/Schools_02.08.2023 16-56.xlsx', dtype = str)
schools_data

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Телефон,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,(0800)330810,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5264358,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,"(044)234-50-55, (044)235-60-63",(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)259-31-60,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)2584242,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,(046)422-43-40,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,(0462)687733,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,(04653)27571,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,(046)5924644,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90


In [23]:
schools_data.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'Код ЄДРПОУ', 'Статус', 'Тип закладу', 'Форма власності', 'Код КОАТУУ',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги'],
      dtype='object')

In [24]:
# create new with id
schools_data_id = schools_data.copy()
create_id(schools_data_id, 'Повна назва', 'id')

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...


In [25]:
schools_data_id.rename(columns={'Код КОАТУУ':'KOATUU_2020', 'Код ЄДРПОУ':'EDRPOU'}, inplace=True)
schools_data_id.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'EDRPOU', 'Статус', 'Тип закладу', 'Форма власності', 'KOATUU_2020',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги', 'id'],
      dtype='object')

In [26]:
schools_data_id[schools_data_id.KOATUU_2020.str.len() != 10]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id


In [27]:
schools_data_id.EDRPOU = schools_data_id.EDRPOU.apply(lambda x: x.zfill(8) if isinstance(x, str) and len(x) >= 6 else None)

In [28]:
schools_data_id[(schools_data_id.EDRPOU.str.len() < 8)&(schools_data_id.EDRPOU.str.len().notna())]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id


In [29]:
schools_data_id.loc[:,'KOATUU_2020_reg'] = schools_data_id.loc[:,'KOATUU_2020'].str[:2]
schools_data_id

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки,80
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив,80
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив,80
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив,80
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...,74
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...,74
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...,74
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...,74


In [30]:
schools_data_id = schools_data_id[schools_data_id.EDRPOU != '02301958']
schools_data_id

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки,80
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив,80
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив,80
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив,80
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...,74
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...,74
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...,74
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...,74


In [31]:
schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
1611,Велимченський ліцей Велимченської сільської ра...,176569,так,Велимченський ліцей,21747814,ліквідовано,ліцей,Комунальна,0724280401,Волинська область,...,NaN,NaN,Директор Павлович Тетяна Володимирівна,ні,ні,ні,ні,NaN,велимченськийлцейвелимченськаволинськаобл,07
1612,Велимченський ліцей Велимченської сільської ра...,143856,так,Велимченський ліцей,21747814,працює,ліцей,Комунальна,0724280401,Волинська область,...,velimche__shcola@ukr.net,http://velimcheshkola.at.ua/,Павлович Тетяна Володимирівна,ні,так,ні,ні,NaN,велимченськийлцейвелимченськаволинськаобл,07
10031,Трикратська ЗОШ І-ІІІ ступенів Олександрівсько...,151531,так,NaN,None,працює,школа,Комунальна,4822085001,Миколаївська область,...,a_grigorivna@ukr.net,https://trykraty.e-schools.info/,Директор Багнюк Віталій Михайлович,ні,так,ні,ні,220,трикратськазагальноосвтняшкола3ступолександрвс...,48
10032,Трикратська ЗОШ І-ІІІ ступенів Олександрівсько...,151533,так,Трикратська ЗОШ І-ІІІ ступенів,None,працює,школа,Комунальна,4822085001,Миколаївська область,...,a_grigorivna@ukr.net,https://trykraty.e-schools.info/,Директор Багнюк Віталій Михайлович,ні,так,ні,ні,220,трикратськазагальноосвтняшкола3ступолександрвс...,48
14862,Подо-Калинівський ЗЗСО І-ІІІ ст,151577,так,NaN,None,працює,школа,Комунальна,6525082501,Херсонська область,...,kalina19802@ukr.net,http://www.kalina1980.ucoz.ua,Директор Маляренко Віктор Олександрович,ні,так,ні,ні,464,подокалинвський3с,65
14863,Подо-Калинівський ЗЗСО І-ІІІ ст,151582,так,NaN,None,працює,школа,Комунальна,6525082501,Херсонська область,...,kalina19802@ukr.net,http://www.kalina1980.ucoz.ua,Директор Маляренко Віктор Олександрович,ні,так,ні,ні,464,подокалинвський3с,65
14909,Старо-Маячківський ЗЗСО І ст,151578,так,NaN,None,працює,початкова школа,Комунальна,6525082502,Херсонська область,...,stara-mayachka@ukr.net,NaN,Директор Несторяк Наталія Іванівна,ні,так,ні,ні,40,старомаячквськийс,65
14910,Старо-Маячківський ЗЗСО І-ст,151583,так,NaN,None,працює,початкова школа,Комунальна,6525082502,Херсонська область,...,stara-mayachka@ukr.net,NaN,Директор Несторяк Наталія Іванівна,ні,так,ні,ні,40,старомаячквськийс,65


In [32]:
schools_data_id = schools_data_id.drop_duplicates(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep='last')

In [33]:
schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
91,"Приватний заклад освіти ""УОЛД СКУЛ КАМПУС А""",151623,так,"ПЗО ""УОЛД СКУЛ КАМПУС А""",42180923,працює,школа,Приватна,8036300000,КИЇВ,...,campus.a@worldschool.com.ua,NaN,Директор Бадаєва Інна Миколаївна,ні,ні,ні,ні,NaN,приватнийзакладосвтиуолдскулкампус,80
254,Технічний ліцей м. Києва,145237,так,Технічний ліцей м. Києва,22876023,працює,ліцей,Комунальна,8036600000,КИЇВ,...,dtl-licej@ukr.ua,http://tl.kyiv.ua/,Директор Андрієнко Анатолій Михайлович,ні,ні,ні,ні,NaN,технчнийлцеймкив,80
268,"Приватний заклад освіти ""УОЛД СКУЛ КАМПУС Б""",176647,так,"ПЗО ""УОЛД СКУЛ КАМПУС Б""",43594721,працює,школа,Приватна,8000000000,КИЇВ,...,NaN,NaN,Директор Давидова Наталія Леонідівна,ні,ні,ні,ні,NaN,приватнийзакладосвтиуолдскулкампус,80
584,Технічний ліцей Шевченківського району м. Києва,149078,так,Технічний ліцей,22881946,працює,ліцей,Комунальна,8039100000,КИЇВ,...,texlyceum@ukr.net,techlyceum.kiev.ua,Директор Ілюшина Олена Олександрівна,ні,ні,ні,ні,844,технчнийлцеймкив,80
4728,Данилівський заклад загальної середньої освіти...,176502,так,Данилівський НВК І-ІІІ ступенів,40760974,працює,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,171,данилвський3ступхустськазакарпатськаобл,21
4729,Данилівський заклад загальної середньої освіти...,148256,так,ДАНИЛІВСЬКИЙ ЗЗСО І-ІІІ СТУПЕНІВ,34440724,реорганізовано,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,NaN,данилвський3ступхустськазакарпатськаобл,21
6818,Гоголівський ліцей Великодимерської селищної р...,146689,так,Гоголівський ліцей,25299425,працює,ліцей,Комунальна,3221281601,Київська область,...,gogoliv.school@gmail.com,http://gogolivska-zosh.kiev.sch.in.ua/,Директор Дяченко Оксана Іванівна,Опорний заклад,так,ні,ні,NaN,гоголвськийлцейвеликодимерськакивськаобл,32
6819,Гоголівський ліцей Великодимерської селищної р...,147130,так,філія Гоголівського ліцею,25299342,працює,ліцей,Комунальна,3221281601,Київська область,...,gogoliv-school2@ukr.net,http://gogoliv-school2.kiev.sch.in.ua,Завідувач філією Мишкоріз Любов Миколаївна,Філія,так,ні,ні,NaN,гоголвськийлцейвеликодимерськакивськаобл,32
11534,"Комунальний заклад ""Розсошенська гімназія Щерб...",139999,так,Розсошенська гімназія,21071866,працює,гімназія,Комунальна,5324087705,Полтавська область,...,rzsgymn@gmail.com,http://rzsschool.org.ua/,Директор Яловець Євген Олександрович,ні,так,ні,ні,834,розсошенськагмназящербанвськаполтавськаобл,53
11535,"Комунальний заклад ""Розсошенська гімназія Щерб...",151527,так,Розсошенська гімназія,52852882,ліквідовано,гімназія,Комунальна,5324087705,Полтавська область,...,rzsgymn@gmail.com,http://rzsschool.ucoz.ua/,Директор Яловець Євген Олександрович,ні,так,ні,ні,NaN,розсошенськагмназящербанвськаполтавськаобл,53


In [34]:
schools_data_id = schools_data_id[~schools_data_id.EDRPOU.isin(['34440724', '52852882', '58826264', 
                                                                '33329415', '26271178', '27646632', 
                                                                '04393999', '24623237', '25649174',
                                                                '25649172', '21977976', '26213928',
                                                                '06716337', '26291911'])]

In [35]:
schools_data_id.drop(schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)

In [36]:
dataset_2020_mon= dataset_2020.merge(schools_data_id[['id', 'KOATUU_2020_reg', 'EDRPOU']], on=['id', 'KOATUU_2020_reg'], how='left')
dataset_2020_mon

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,NaN,02553000000,комунальна,7137,25844555,2020,05,,загальноосвтнйнавчальновиховнийкомплекс3ступшк...,NaN
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,NaN,02553000000,комунальна,6936,25844992,2020,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,NaN,02553000000,комунальна,7149,25844928,2020,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,NaN,02553000000,комунальна,7191,25844970,2020,05,,загальноосвтнясередняшкола3ступ5козятинськавни...,NaN
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,NaN,02553000000,комунальна,6976,23101028,2020,05,,комунальнийнавчальнийзакладспецалзованашкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,NaN,26000000000,комунальна,22883,23392178,2020,80,,гмназя237,23392178
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,NaN,26000000000,комунальна,23029,23392149,2020,80,,середнязагальноосвтняшкола3ступ195менвкудряшов...,23392149
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,NaN,26000000000,комунальна,23280,26187881,2020,80,,спецалзованашкола3ступ73зпоглибленимвивченямук...,26187881
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,NaN,26000000000,комунальна,23154,22878447,2020,80,,спецалзованашкола131мкив,22878447


In [37]:
dataset_2020_mon.loc[(dataset_2020_mon['EDRPOU_y'].notna())&(dataset_2020_mon.EDRPOU_checked == ''), 'EDRPOU_checked'] = dataset_2020_mon.loc[(dataset_2020_mon['EDRPOU_y'].notna())&(dataset_2020_mon.EDRPOU_checked == ''), 'EDRPOU_y']

In [38]:
dataset_2020_mon = dataset_2020_mon.drop(columns=['EDRPOU_y'], axis=1)
dataset_2020_mon = dataset_2020_mon.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [39]:
dataset_2020_mon[dataset_2020_mon['EDRPOU_checked'] != '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
13,Вінницька область,місто,Могилів-Подільська міська ТГ,Гімназія № 2 м.Могилева-Подільського Вінницько...,I-III,NaN,51,26,4,713,...,16310,NaN,02558000000,комунальна,11323,43125858,2020,05,43125858,гмназя2ммогилеваподльськоговницькаобл
114,Вінницька область,місто,Турбівська селищна ТГ,Турбівська загальноосвітня школа І-ІІІ ст. №2 ...,I-III,NaN,39,17,3,485,...,11977.156,NaN,02568000000,комунальна,11405,21723820,2020,05,21723820,турбвськазагальноосвтняшкола3ступ2х000dлиповец...
120,Вінницька область,село,Іванівська сільська ТГ,Середня загальноосвітня школа І - ІІІ ступенів...,I-III,NaN,23,18,8,195,...,8302,NaN,02526000000,комунальна,7995,26229088,2020,05,26229088,середнязагальноосвтняшкола3ступсуладвкавницькаобл
124,Вінницька область,село,Літинська селищна ТГ,Середня загальноосвітня школа І-ІІІ ступенів с...,I-III,NaN,17,18,1,112,...,5545.8,NaN,02541000000,комунальна,7988,26223660,2020,05,26223660,середнязагальноосвтняшкола3ступсвч
144,Вінницька область,село,Мурованокуриловецька селищна ТГ,"Навчально виховний комплекс ""Загальноосвітній ...",I-III,NaN,13,12,2,44,...,2823.2,NaN,02559000000,комунальна,6399,26336200,2020,05,26336200,навчальновиховнийкомплексзагальноосвтнйнавчаль...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив


### short name

In [40]:
schools_data_id_short = schools_data_id[schools_data_id['Скорочена назва'].notna()]

In [41]:
create_id(schools_data_id_short, 'Скорочена назва', 'id')

/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[id_] = dataset[id_].apply(del_rayon)
/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[id_] = dataset[id_].apply(lambda s: s if (len(s) and s[

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки,80
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236,80
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87,80
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150,80
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,"комунальний заклад ""Яблунівський ліцей"" Черніг...",139767,так,"КЗ ""Яблунівський ліцей""",05266240,працює,ліцей,Комунальна,7424189701,Чернігівська область,...,Yablunovka@ukr.net,NaN,Директор Шаповаленко Неоніла Олександрівна,Опорний заклад,так,ні,так,170,яблунвськийлце,74
17329,Яблунівська загальноосвітня школа I-III ступен...,144175,так,Яблунівська ЗОШ І-ІІІ ступенів,33208204,працює,школа,Комунальна,7424189701,Чернігівська область,...,yablunivkask@ukr.net,NaN,Директор Колесник Анатолій Васильович,ні,так,ні,ні,320,яблунвськазагальноосвтняшкола3ступен,74
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3сту,74
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2сту,74


In [42]:
schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg


In [43]:
schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
28,Спеціалізована школа І-ІІІ ступенів з поглибле...,148398,так,спеціалізована школа,22877005,працює,спеціалізована школа,Комунальна,8036100000,КИЇВ,...,sch15@i.ua,school15.kyiv.ua,Директор Литвин Валерій Олексійович,ні,ні,ні,ні,NaN,спецалзованашкол,80
496,ліцей №142 м. Києва,147801,так,ліцей,21622214,працює,ліцей,Комунальна,8038900000,КИЇВ,...,l142@ukr.net,NaN,Директор Сільвестрова Ірина Анатоліївна,ні,ні,ні,ні,600,лце,80
547,юридичний ліцей імені Ярослава Кондратьєва Нац...,147605,так,ліцей,35008569,працює,ліцей-інтернат,Державна,8038900000,КИЇВ,...,licei@naiau.kiev.ua,licey.naiau.kiev.ua,Ректор Чернєй Володимир Васильович,ні,ні,ні,ні,200,лце,80
572,Спеціалізована школа І-ІІІ ступенів №28 з погл...,149079,так,спеціалізована школа,22880792,працює,спеціалізована школа,Комунальна,8039100000,КИЇВ,...,school28kiev@ukr.net,sites.google.com/view/sch28,Директор Охріменко Наталія Миколаївна,ні,ні,ні,ні,1176,спецалзованашкол,80
652,Обласний науковий ліцей-інтернат Комунального ...,143380,так,Обласний науковий ліцей-інтернат,36364955,працює,науковий ліцей-інтернат,Комунальна,0510100000,Вінницька область,...,vinlitsey@ukr.net,https://www.vgpk.com.ua,Директор Войцехівський Костянтин Францович,ні,ні,ні,так,720,обласнийнауковийлцейнтерна,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15586,Рудська гімназія Смотрицької селищної ради Кам...,141431,так,Рудська гімназія,22777640,працює,гімназія,Комунальна,6821887701,Хмельницька область,...,rud.zosh2015@ukr.net,http://www.rudka-school.at.ua,Директор Середа Валентина Олексіївна,ні,так,ні,ні,NaN,рудськагмназ,68
16194,Рижавська філія Ладижинського ліцею Ладижинськ...,134934,так,Рижавська філія,24352264,працює,школа,Комунальна,7124387301,Черкаська область,...,rugavkaschool@gmail.com,NaN,Завідувач філією Квашук Оксана Миколаївна,Філія,ні,ні,ні,1,рижавськафл,71
16195,Рижавська філія Ладижинської загальноосвітньої...,151388,так,Рижавська філія,42469234,працює,школа,Комунальна,7124387301,Черкаська область,...,rugavkaschool@gmail.com,NaN,Завідувач філією Квашук Оксана Миколаївна,ні,так,ні,ні,1,рижавськафл,71
16551,Кам'янський ліцей Кам'янської сільської ради Ч...,142265,так,Кам'янський ліцей,21440097,працює,ліцей,Комунальна,7324584001,Чернівецька область,...,Kamjana-shkola@meta.ua,NaN,Директор Кожелянко Світлана Михайлівна,ні,так,ні,ні,550,камянськийлце,73


In [44]:
schools_data_id_short.drop(schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_8256/2976488812.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_data_id_short.drop(schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)


In [45]:
dataset_2020_mon_short= dataset_2020_mon.merge(schools_data_id_short[['id', 'KOATUU_2020_reg', 'EDRPOU']], on=['id', 'KOATUU_2020_reg'], how='left')
dataset_2020_mon_short

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,NaN,02553000000,комунальна,7137,25844555,2020,05,,загальноосвтнйнавчальновиховнийкомплекс3ступшк...,NaN
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,NaN,02553000000,комунальна,6936,25844992,2020,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,NaN,02553000000,комунальна,7149,25844928,2020,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,NaN,02553000000,комунальна,7191,25844970,2020,05,,загальноосвтнясередняшкола3ступ5козятинськавни...,NaN
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,NaN,02553000000,комунальна,6976,23101028,2020,05,,комунальнийнавчальнийзакладспецалзованашкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237,NaN
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...,NaN
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...,NaN
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив,NaN


In [46]:
dataset_2020_mon_short.loc[(dataset_2020_mon_short['EDRPOU_y'].notna())&(dataset_2020_mon_short.EDRPOU_checked == ''), 'EDRPOU_checked'] = dataset_2020_mon_short.loc[(dataset_2020_mon_short['EDRPOU_y'].notna())&(dataset_2020_mon_short.EDRPOU_checked == ''), 'EDRPOU_y']

In [47]:
dataset_2020_mon_short = dataset_2020_mon_short.drop(columns=['EDRPOU_y'], axis=1)
dataset_2020_mon_short = dataset_2020_mon_short.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [48]:
dataset_2020_mon_short[dataset_2020_mon_short['EDRPOU_checked'] != '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
13,Вінницька область,місто,Могилів-Подільська міська ТГ,Гімназія № 2 м.Могилева-Подільського Вінницько...,I-III,NaN,51,26,4,713,...,16310,NaN,02558000000,комунальна,11323,43125858,2020,05,43125858,гмназя2ммогилеваподльськоговницькаобл
114,Вінницька область,місто,Турбівська селищна ТГ,Турбівська загальноосвітня школа І-ІІІ ст. №2 ...,I-III,NaN,39,17,3,485,...,11977.156,NaN,02568000000,комунальна,11405,21723820,2020,05,21723820,турбвськазагальноосвтняшкола3ступ2х000dлиповец...
120,Вінницька область,село,Іванівська сільська ТГ,Середня загальноосвітня школа І - ІІІ ступенів...,I-III,NaN,23,18,8,195,...,8302,NaN,02526000000,комунальна,7995,26229088,2020,05,26229088,середнязагальноосвтняшкола3ступсуладвкавницькаобл
124,Вінницька область,село,Літинська селищна ТГ,Середня загальноосвітня школа І-ІІІ ступенів с...,I-III,NaN,17,18,1,112,...,5545.8,NaN,02541000000,комунальна,7988,26223660,2020,05,26223660,середнязагальноосвтняшкола3ступсвч
144,Вінницька область,село,Мурованокуриловецька селищна ТГ,"Навчально виховний комплекс ""Загальноосвітній ...",I-III,NaN,13,12,2,44,...,2823.2,NaN,02559000000,комунальна,6399,26336200,2020,05,26336200,навчальновиховнийкомплексзагальноосвтнйнавчаль...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив


## youcontrol

In [49]:
youcontrol =  pd.read_excel('./school_data/Youcontrol_historical_name.xlsx', dtype='str')
youcontrol 

,Код ЄДРПОУ,Назва,Коротка назва
0,00121146,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...",NaN
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ"""
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ"""
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN
...,...,...,...
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""
80924,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""


In [50]:
youcontrol[youcontrol.duplicated(subset=['Назва', 'Код ЄДРПОУ'], keep=False)]

,Код ЄДРПОУ,Назва,Коротка назва
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN
5,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",КП МК-26
119,00452713,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЗВЕНИГОРОДСЬКИЙ Ц...","ДНЗ ""ЗВЕНИГОРОДСЬКИЙ ЦППРК"""
121,00452713,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЗВЕНИГОРОДСЬКИЙ Ц...","ДНЗ ""ЗВЕНИГОРОДСЬКИЙ ЦППРК"""
144,00854423,"КООПЕРАТИВ ""НОВОКРОПИВНИЦЬКИЙ""",NaN
...,...,...,...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80919,60043961,ДЕРЕБЧИНСЬКА СЕРЕДНЯ ЗАГАЛЬНООСВІТНЯ ШКОЛА I-I...,ДЕРЕБЧИНСЬКА СЗШ I-III СТУПЕНІВ
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""


In [51]:
youcontrol_long = youcontrol.drop_duplicates(subset=['Назва', 'Код ЄДРПОУ'], keep='first')

In [52]:
create_id(youcontrol_long, 'Назва', 'id')

/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,id_] = dataset[attr]
/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[id_] = dataset[id_].apply(del_rayon)
/Users/scipyguru/Library/Mob

,Код ЄДРПОУ,Назва,Коротка назва,id
0,00121146,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...",NaN,вдкритеакцонернетовариствопвдентеплоенергомонта
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",публчнеакцонернетовариствопвдентеплоенергомонта
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",приватнеакцонернетовариствопвдентеплоенергомонта
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN,колективнепдпримствоблгородднстровськамеханзов...
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN,колективнепдпримствоблгородднстровськамеханзов...
...,...,...,...,...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорний3ступвницькаобл
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорний3ступвницькаобл
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступджуринськавниць...
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступджуринськавниць...


In [53]:
youcontrol_long.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_8256/2371828141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_long.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)


In [54]:
youcontrol_long[(youcontrol_long.EDRPOU.str.len() < 8)&(youcontrol_long.EDRPOU.str.len().notna())]

,EDRPOU,Назва,Коротка назва,id


In [55]:
youcontrol_long[youcontrol_long.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN,колективнепдпримствоблгородднстровськамеханзов...
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN,колективнепдпримствоблгородднстровськамеханзов...
48,00192471,"КОЛЕКТИВНЕ СПЕЦІАЛІЗОВАНЕ ПІДПРИЄМСТВО ""УКРЕНЕ...",NaN,колективнеспецалзованепдпримствоукренергочормет2
49,00192471,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ СП...,NaN,колективнеспецалзованепдпримствоукренергочормет2
67,00290280,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""НОВООЛЕКСАНДРІ...",NaN,вдкритеакцонернетовариствоновоолександрвськийц...
...,...,...,...,...
80917,60043961,ДЕРЕБЧИНСЬКА СЕРЕДНЯ ЗАГАЛЬНООСВІТНЯ ШКОЛА I-I...,ДЕРЕБЧИНСЬКА СЗШ I-III СТУПЕНІВ,деребчинськасереднязагальноосвтняшкола3ступвни...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорний3ступвницькаобл
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорний3ступвницькаобл
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступджуринськавниць...


In [56]:
youcontrol_long = youcontrol_long.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [57]:
youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
221,01284979,МІЖШКІЛЬНИЙ НАВЧАЛЬНО-ВИРОБНИЧИЙ КОМБІНАТ,МНВК,мжшкльнийнавчальновиробничийкомбна
494,02218789,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) МУЗИ...",NaN,музичнашкола2
498,02218884,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) КОМУ...",NaN,музичнашкола3
503,02218921,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) КОМУ...",NaN,музичнашкола2
719,02542129,МИКОЛАЇВСЬКИЙ ПРОФЕСІЙНИЙ ЛІЦЕЙ,NaN,миколавськийпрофесйнийлце
...,...,...,...,...
80889,45117367,"ФЕРМЕРСЬКЕ ГОСПОДАРСТВО ""ДОВІРА Б""","ФГ ""ДОВІРА Б""",фермерськегосподарстводовра
80903,45251511,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназязпочатковоюшколоютадошкльнимпдроз...
80905,45275530,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназязпочатковоюшколоютадошкльнимпдроз...
80907,45287518,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашколаздошкльнимпдроздломс...


In [58]:
youcontrol_long.drop(youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_8256/2842222147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_long.drop(youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)


In [59]:
dataset_2020_youcontrol_long= dataset_2020_mon_short.merge(youcontrol_long[['id', 'EDRPOU']], on=['id'], how='left')
dataset_2020_youcontrol_long

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,NaN,02553000000,комунальна,7137,25844555,2020,05,,загальноосвтнйнавчальновиховнийкомплекс3ступшк...,25844555
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,NaN,02553000000,комунальна,6936,25844992,2020,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...,25844992
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,NaN,02553000000,комунальна,7149,25844928,2020,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...,25844928
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,NaN,02553000000,комунальна,7191,25844970,2020,05,,загальноосвтнясередняшкола3ступ5козятинськавни...,NaN
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,NaN,02553000000,комунальна,6976,23101028,2020,05,,комунальнийнавчальнийзакладспецалзованашкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237,23392178
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...,NaN
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...,26187881
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив,22878447


In [60]:
dataset_2020_youcontrol_long.loc[(dataset_2020_youcontrol_long['EDRPOU_y'].notna())&(dataset_2020_youcontrol_long.EDRPOU_checked == ''), 'EDRPOU_checked'] = dataset_2020_youcontrol_long.loc[(dataset_2020_youcontrol_long['EDRPOU_y'].notna())&(dataset_2020_youcontrol_long.EDRPOU_checked == ''), 'EDRPOU_y']

In [61]:
dataset_2020_youcontrol_long = dataset_2020_youcontrol_long.drop(columns=['EDRPOU_y'], axis=1)
dataset_2020_youcontrol_long = dataset_2020_youcontrol_long.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [62]:
dataset_2020_youcontrol_long[dataset_2020_youcontrol_long['EDRPOU_checked'] != '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,5006.502,NaN,02553000000,комунальна,7137,25844555,2020,05,25844555,загальноосвтнйнавчальновиховнийкомплекс3ступшк...
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,8614.601,NaN,02553000000,комунальна,6936,25844992,2020,05,25844992,комунальнийнавчальнийзакладзагальноосвтняшкола...
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,12627.197,NaN,02553000000,комунальна,7149,25844928,2020,05,25844928,комунальнийнавчальнийзакладзагальноосвтняшкола...
5,Вінницька область,місто,Козятинська міська ТГ,"Комунальний заклад ""Навчально-виховний комплек...",I-III,NaN,58,27,4,820,...,21163.339,NaN,02553000000,комунальна,22498,25512362,2020,05,38986337,навчальновиховнийкомплекс3ступлцейшколакозятин...
6,Вінницька область,місто,Ладижинська міська ТГ,Ладижинський навчально-виховний комплекс №1 (з...,I-III,NaN,20,18,4,239,...,9855,NaN,02556000000,комунальна,7808,26219581,2020,05,26219581,ладижинськийнавчальновиховнийкомплекс1ладижинс...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив


## youcontrol short

In [63]:
youcontrol_short = youcontrol[youcontrol['Коротка назва'].notna()]

In [64]:
youcontrol_short.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_8256/692976443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_short.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)


In [65]:
create_id(youcontrol_short, 'Коротка назва', 'id')

/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,id_] = dataset[attr]
/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[id_] = dataset[id_].apply(del_rayon)
/Users/scipyguru/Library/Mob

,EDRPOU,Назва,Коротка назва,id
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
5,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",КП МК-26,кпмк26
7,00131564,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПРИКАРПАТТЯОБЛ...","ПАТ ""ПРИКАРПАТТЯОБЛЕНЕРГО""",патприкарпаттяобленерг
8,00131564,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПРИКАРПАТТЯОБЛ...","АТ ""ПРИКАРПАТТЯОБЛЕНЕРГО""",атприкарпаттяобленерг
...,...,...,...,...
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,доззс
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступен
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступен
80924,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступен


In [66]:
youcontrol_short[youcontrol_short.duplicated(subset=['Коротка назва', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
11,00131954,"АКЦІОНЕРНА ТОВАРИСТВО ""ХАРКІВОБЛЕНЕРГО""","АТ ""ХАРКІВОБЛЕНЕРГО""",атхарквобленерг
12,00131954,"АКЦІОНЕРНЕ ТОВАРИСТВО ""ХАРКІВОБЛЕНЕРГО""","АТ ""ХАРКІВОБЛЕНЕРГО""",атхарквобленерг
62,00220434,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""НАУКОВО-ВИРОБН...","АТ ""НВАТ ""ВНДІКОМПРЕСОРМАШ""",атнватвндкомпресорма
...,...,...,...,...
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,доззс
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,доззс
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступен
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступен


In [67]:
youcontrol_short = youcontrol_short.drop_duplicates(subset=['Коротка назва', 'EDRPOU'], keep='first')

In [68]:
youcontrol_short[youcontrol_short.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
103,00389357,ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ЛЬВІВСЬКЕ ВИЩЕ ПРО...,ДНЗ ЛЬВІВСЬКЕ ВПУХТ,днзльввськевпух
104,00389357,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЛЬВІВСЬКЕ ВИЩЕ ПР...","ДНЗ ""ЛЬВІВСЬКЕ ВПУХТ""",днзльввськевпух
174,00951592,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ЄЛИЗАВЕТІВСЬК...","ПРАТ ""ЄЛИЗАВЕТІВСЬКИЙ ЕЛЕВАТОР""",пратлизаветвськийелевато
175,00951592,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ЄЛИЗАВЕТІВСЬКИ...","ПРАТ ""ЄЛИЗАВЕТІВСЬКИЙ ЕЛЕВАТОР""",пратлизаветвськийелевато
199,01204408,"""ПОЛТАВСЬКИЙ ОБЛАСНИЙ САНАТОРІЙ ДЛЯ ДІТЕЙ З ПО...","КП"" ПОДС""",кппод
...,...,...,...,...
80431,43861328,"НАЦІОНАЛЬНИЙ УНІВЕРСИТЕТ ""ОДЕСЬКА ПОЛІТЕХНІКА""",ОДЕСЬКА ПОЛІТЕХНІКА,одеськаполтехнк
80518,44129392,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ПРИВАТ...,"ТОВ ПЗ ""ХЛ ""ХАНІ АКАДЕМІЯ"" ХО",пзхлханакадемях
80519,44129392,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ПРИВАТ...,"ТОВ ПЗ ""ХЛ ""ХАНІ АКАДЕМІЯ"""" ХО",пзхлханакадемях
80581,44271671,РУБІЖАНСЬКИЙ ЛІЦЕЙ РУБІЖАНСЬКОЇ МІСЬКОЇ РАДИ Л...,РУБІЖАНСЬКИЙ ЛІЦЕЙ,рубжанськийлце


In [69]:
youcontrol_short = youcontrol_short.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [70]:
youcontrol_short[youcontrol_short.duplicated(subset=['id'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
221,01284979,МІЖШКІЛЬНИЙ НАВЧАЛЬНО-ВИРОБНИЧИЙ КОМБІНАТ,МНВК,мнв
295,01489345,ЛІЦЕЙ №18 ДЕСНЯНСЬКОГО РАЙОНУ МІСТА КИЄВА,ЛІЦЕЙ №18,лцей18
332,01730137,ОЛЕКСАНДРІВСЬКИЙ КОМБІНАТ КООПЕРАТИВНОЇ ПРОМИС...,ОЛЕКСАНДРІВСЬКИЙ ККП,олександрвськийкк
418,02137051,Ліцей с. Стрілки Стрілківської сільської ради ...,Стрілківський ліцей,стрлквськийлце
510,02218967,"КОМУНАЛЬНИЙ ЗАКЛАД ""МИСТЕЦЬКА ШКОЛА №2 КРАМАТО...",МИСТЕЦЬКА ШКОЛА № 2,мистецькашкола2
...,...,...,...,...
80905,45275530,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназ
80906,45277868,"Комунальний заклад ""Петрівська гімназія"" Кодим...","КЗ ""Петрівська гімназія""",петрвськагмназ
80907,45287518,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашкол
80912,45383386,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашкол


In [71]:
youcontrol_short.drop(youcontrol_short[youcontrol_short.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

In [72]:
dataset_2020_youcontrol_short= dataset_2020_youcontrol_long.merge(youcontrol_short[['id', 'EDRPOU']], on=['id'], how='left')
dataset_2020_youcontrol_short

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,NaN,02553000000,комунальна,7137,25844555,2020,05,25844555,загальноосвтнйнавчальновиховнийкомплекс3ступшк...,NaN
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,NaN,02553000000,комунальна,6936,25844992,2020,05,25844992,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,NaN,02553000000,комунальна,7149,25844928,2020,05,25844928,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,NaN,02553000000,комунальна,7191,25844970,2020,05,,загальноосвтнясередняшкола3ступ5козятинськавни...,NaN
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,NaN,02553000000,комунальна,6976,23101028,2020,05,,комунальнийнавчальнийзакладспецалзованашкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237,23392178
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...,NaN
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...,NaN
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив,NaN


In [73]:
dataset_2020_youcontrol_short.loc[(dataset_2020_youcontrol_short['EDRPOU_y'].notna())&(dataset_2020_youcontrol_short.EDRPOU_checked == ''), 'EDRPOU_checked'] = dataset_2020_youcontrol_short.loc[(dataset_2020_youcontrol_short['EDRPOU_y'].notna())&(dataset_2020_youcontrol_short.EDRPOU_checked == ''), 'EDRPOU_y']

In [74]:
dataset_2020_youcontrol_short = dataset_2020_youcontrol_short.drop(columns=['EDRPOU_y'], axis=1)
dataset_2020_youcontrol_short = dataset_2020_youcontrol_short.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [75]:
dataset_2020_youcontrol_short[dataset_2020_youcontrol_short['EDRPOU_checked'] != '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,5006.502,NaN,02553000000,комунальна,7137,25844555,2020,05,25844555,загальноосвтнйнавчальновиховнийкомплекс3ступшк...
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,8614.601,NaN,02553000000,комунальна,6936,25844992,2020,05,25844992,комунальнийнавчальнийзакладзагальноосвтняшкола...
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,12627.197,NaN,02553000000,комунальна,7149,25844928,2020,05,25844928,комунальнийнавчальнийзакладзагальноосвтняшкола...
5,Вінницька область,місто,Козятинська міська ТГ,"Комунальний заклад ""Навчально-виховний комплек...",I-III,NaN,58,27,4,820,...,21163.339,NaN,02553000000,комунальна,22498,25512362,2020,05,38986337,навчальновиховнийкомплекс3ступлцейшколакозятин...
6,Вінницька область,місто,Ладижинська міська ТГ,Ладижинський навчально-виховний комплекс №1 (з...,I-III,NaN,20,18,4,239,...,9855,NaN,02556000000,комунальна,7808,26219581,2020,05,26219581,ладижинськийнавчальновиховнийкомплекс1ладижинс...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив


# collected with bot

In [76]:
data_bot = pd.read_csv('./fin_data/checked_links_2020.csv', dtype = str, names = ['eoname', 'url', 'EDRPOU'])
data_bot

,eoname,url,EDRPOU
0,Загальноосвітній навчально-виховний комплекс І...,https://opendatabot.ua/c/34377349,Not found.
1,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",https://opendatabot.ua/c/34377473,Not found.
2,"Комунальний навчальний заклад ""Загальноосвітня...",https://opendatabot.ua/c/34377349,Not found.
3,Загальноосвітня середня школа І-ІІІ ступенів №...,https://opendatabot.ua/c/25844970,Not found.
4,"Мальчовецький навчально-виховний комплекс ""Дош...",https://opendatabot.ua/c/26233664,26233664
...,...,...,...
10003,"Навчально-виховний комплекс ""дошкільний навчал...",https://opendatabot.ua/c/22879240,Not found.
10004,"НВК ""Спеціалізована школа І ступеня з поглибле...",https://opendatabot.ua/c/21580289,21580289
10005,Початкова школа № 334 Дарницького району м. Києва,https://opendatabot.ua/c/42384408,Not found.
10006,Школа І ступеня № 333 Дарницького району м. Києва,https://opendatabot.ua/c/41659088,41659088


In [77]:
data_bot[(data_bot.EDRPOU.str.len() < 8)&(data_bot.EDRPOU.str.len().notna())]

,eoname,url,EDRPOU
383,Загальноосвітня школа І-ІІ ступеня с. Жабче Го...,https://opendatabot.ua/c/02141710,2141710
1565,Лиманська загальноосвітня школа І-ІІІ ступенів...,https://opendatabot.ua/c/04053275,4053275
1803,СТРІЄВСЬКА ЗАГАЛЬНООСВІТНЯ ШКОЛА І - ІІ СТУПЕН...,https://opendatabot.ua/c/06670109,6670109
1804,ІВАШКІВСЬКА ЗАГАЛЬНООСВІТНЯ ШКОЛА І - ІІ СТУПЕ...,https://opendatabot.ua/c/06669974,6669974
1807,Ігнатпільська загальноосвітня школа І-ІІІ ступ...,https://opendatabot.ua/c/06670664,6670664
1809,Левковицька загальноосвітня школа I-II ступені...,https://opendatabot.ua/c/06670865,6670865
1812,Гладковицька загальноосвітня школа І-ІІІ ступе...,https://opendatabot.ua/c/06670919,6670919
1828,Дівочківська загальноосвітня школа І-ІІ ступен...,https://opendatabot.ua/c/06671333,6671333
1830,Горбулівська загальноосвітня школа І-ІІІ ступе...,https://opendatabot.ua/c/06671445,6671445
1835,Андріївська загальноосвітня школа І-ІІІ ступен...,https://opendatabot.ua/c/06671422,6671422


In [78]:
data_bot['EDRPOU']=data_bot['EDRPOU'].str.zfill(8)

In [79]:
data_bot.drop_duplicates(inplace=True)

In [80]:
data_bot[data_bot.duplicated(subset=['eoname'], keep=False)]

,eoname,url,EDRPOU


In [81]:
data_bot.drop(data_bot[data_bot.duplicated(subset=['eoname'], keep=False)].index.values.tolist(), inplace=True)

In [82]:
dataset_2020_bot= dataset_2020_youcontrol_short.merge(data_bot[['eoname', 'EDRPOU']], on=['eoname'], how='left')
dataset_2020_bot

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,NaN,02553000000,комунальна,7137,25844555,2020,05,25844555,загальноосвтнйнавчальновиховнийкомплекс3ступшк...,Not found.
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,NaN,02553000000,комунальна,6936,25844992,2020,05,25844992,комунальнийнавчальнийзакладзагальноосвтняшкола...,Not found.
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,NaN,02553000000,комунальна,7149,25844928,2020,05,25844928,комунальнийнавчальнийзакладзагальноосвтняшкола...,Not found.
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,NaN,02553000000,комунальна,7191,25844970,2020,05,,загальноосвтнясередняшкола3ступ5козятинськавни...,Not found.
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,NaN,02553000000,комунальна,6976,23101028,2020,05,,комунальнийнавчальнийзакладспецалзованашкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237,NaN
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...,NaN
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...,26187881
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив,Not found.


In [83]:
dataset_2020_bot.loc[(dataset_2020_bot['EDRPOU_y'].notna())&(dataset_2020_bot.EDRPOU_checked == 'Not found.'), 'EDRPOU_checked'] = dataset_2020_bot.loc[(dataset_2020_bot['EDRPOU_y'].notna())&(dataset_2020_bot.EDRPOU_checked == 'Not found.'), 'EDRPOU_y']

In [84]:
dataset_2020_bot = dataset_2020_bot.drop(columns=['EDRPOU_y'], axis=1)
dataset_2020_bot = dataset_2020_bot.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [85]:
to_solve=dataset_2020_bot[(dataset_2020_bot['EDRPOU_checked'] == '')|((dataset_2020_bot['EDRPOU_checked'] != dataset_2020_bot['EDRPOU'])&(dataset_2020_bot['EDRPOU_checked'] != ''))]

In [ ]:
# to_solve.to_csv('dataset_2020_issues.csv', index = False)

## solved issues by hand

In [97]:
by_hand = pd.read_csv('./fin_data/dataset_2020_issues_solved.csv', dtype = str)
by_hand

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,11029.117,NaN,02553000000,комунальна,7191,25844970,2020,05,25844970,загальноосвтнясередняшкола3ступ5козятинськавни...
1,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,19559.764,NaN,02553000000,комунальна,6976,23101028,2020,05,23101028,комунальнийнавчальнийзакладспецалзованашкола3с...
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний заклад ""Навчально-виховний комплек...",I-III,NaN,58,27,4,820,...,21163.339,NaN,02553000000,комунальна,22498,25512362,2020,05,38986337,навчальновиховнийкомплекс3ступлцейшколакозятин...
3,Вінницька область,місто,Могилів-Подільська міська ТГ,Гімназія №5 м. Могилева-Подільского Вінницької...,I-III,NaN,31,29,5,428,...,11650.9,NaN,02558000000,комунальна,11326,43126060,2020,05,43126060,гмназя5ммогилеваподльскоговницькаобл
4,Вінницька область,місто,Могилів-Подільська міська ТГ,Навчально-виховний комплекс №3 «загальноосвітн...,I-III,NaN,48,22,4,816,...,15778.3,NaN,02558000000,комунальна,11324,40686503,2020,05,40686503,навчальновиховнийкомплекс3загальноосвтнйнавчал...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,м.Київ,місто,місто Київ,Спеціалізована загальноосвітня школа І-ІІІ сту...,I-III,NaN,61,40,4,1040,...,26257.687,NaN,26000000000,комунальна,22888,37448113,2020,80,23392238,спецалзованазагальноосвтняшкола3ступ274зпоглиб...
1614,м.Київ,місто,місто Київ,Технічний ліцей Шевченківського району міста К...,II-III,NaN,31,26,2,499,...,9769.923,NaN,26000000000,комунальна,23299,22881946,2020,80,22881946,технчнийлцеймкив
1615,м.Київ,місто,місто Київ,"м.Київ. Навчально-виховний комплекс ""Домінанта...",I-III,NaN,112,85,7,1301,...,55619.1,NaN,26000000000,комунальна,23043,22875220,2020,80,22875220,мкивнавчальновиховнийкомплексдомнант
1616,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів №61 з погл...,I-III,NaN,41,28,2,721,...,13213.396,NaN,26000000000,комунальна,23278,22881805,2020,80,22881805,спецалзованашкола3ступ61зпоглибленимвивченямнф...


In [98]:
by_hand[(by_hand.EDRPOU_checked.str.len() < 8)&(by_hand.EDRPOU_checked.str.len().notna())]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id


In [118]:
by_hand[by_hand.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id


In [126]:
dataset_2020_hand= dataset_2020_bot.merge(by_hand[['id', 'EDRPOU', 'EDRPOU_checked']], on=['id', 'EDRPOU'], how='left')
dataset_2020_hand

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked_x,id,EDRPOU_checked_y
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,NaN,02553000000,комунальна,7137,25844555,2020,05,25844555,загальноосвтнйнавчальновиховнийкомплекс3ступшк...,NaN
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,NaN,02553000000,комунальна,6936,25844992,2020,05,25844992,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,NaN,02553000000,комунальна,7149,25844928,2020,05,25844928,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,NaN,02553000000,комунальна,7191,25844970,2020,05,,загальноосвтнясередняшкола3ступ5козятинськавни...,25844970
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,NaN,02553000000,комунальна,6976,23101028,2020,05,,комунальнийнавчальнийзакладспецалзованашкола3с...,23101028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237,NaN
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...,NaN
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...,NaN
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив,NaN


In [127]:
dataset_2020_hand.loc[dataset_2020_hand['EDRPOU_checked_y'].notna(), 'EDRPOU_checked_x'] = dataset_2020_hand.loc[dataset_2020_hand['EDRPOU_checked_y'].notna(), 'EDRPOU_checked_y']
dataset_2020_hand

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked_x,id,EDRPOU_checked_y
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,NaN,02553000000,комунальна,7137,25844555,2020,05,25844555,загальноосвтнйнавчальновиховнийкомплекс3ступшк...,NaN
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,NaN,02553000000,комунальна,6936,25844992,2020,05,25844992,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,NaN,02553000000,комунальна,7149,25844928,2020,05,25844928,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,NaN,02553000000,комунальна,7191,25844970,2020,05,25844970,загальноосвтнясередняшкола3ступ5козятинськавни...,25844970
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,NaN,02553000000,комунальна,6976,23101028,2020,05,23101028,комунальнийнавчальнийзакладспецалзованашкола3с...,23101028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237,NaN
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...,NaN
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...,NaN
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив,NaN


In [128]:
dataset_2020_hand = dataset_2020_hand.drop(columns=['EDRPOU_checked_y'], axis=1)
dataset_2020_hand = dataset_2020_hand.rename(columns = {'EDRPOU_checked_x':'EDRPOU_checked'})

In [129]:
dataset_2020_hand[dataset_2020_hand.EDRPOU_checked == '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
1262,Дніпропетровська область,місто,Юр'ївська селищна ТГ,"Комунальний опорний заклад освіти""Юр'ївська за...",I-III,NaN,42,57,9,393,...,18129.78,опорна,04552000000,комунальна,3038,35061567,2020,12,,освтиюрвськазагальноосвтняшкола3ступменгерояра...
2041,Житомирська область,село,Глибочицька сільська ТГ,комунальний опорний заклад освіти «Глибочицьки...,I-III,NaN,24,11,3,223,...,8954.4,опорна,06544000000,комунальна,16912,22060706,2020,18,,освтиглибочицькийлцейглибочицькажитомирськаобл...
5403,Одеська область,місто,Окнянська селищна ОТГ,"Комунальний опорний заклад освіти ""Окнянська з...",I-III,NaN,72,49,10,870,...,40729.5,опорна,15528000000,комунальна,15154,41494108,2020,51,,освтиокнянськазагальноосвтняшкола3ступх000dокн...


In [130]:
dataset_2020_hand.loc[dataset_2020_hand.EDRPOU_checked == '', 'EDRPOU_checked'] = dataset_2020_hand.loc[dataset_2020_hand.EDRPOU_checked == '', 'EDRPOU']
dataset_2020_hand

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,5006.502,NaN,02553000000,комунальна,7137,25844555,2020,05,25844555,загальноосвтнйнавчальновиховнийкомплекс3ступшк...
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,8614.601,NaN,02553000000,комунальна,6936,25844992,2020,05,25844992,комунальнийнавчальнийзакладзагальноосвтняшкола...
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,12627.197,NaN,02553000000,комунальна,7149,25844928,2020,05,25844928,комунальнийнавчальнийзакладзагальноосвтняшкола...
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,11029.117,NaN,02553000000,комунальна,7191,25844970,2020,05,25844970,загальноосвтнясередняшкола3ступ5козятинськавни...
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,19559.764,NaN,02553000000,комунальна,6976,23101028,2020,05,23101028,комунальнийнавчальнийзакладспецалзованашкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив


In [131]:
dataset_2020_hand[dataset_2020_hand.EDRPOU != dataset_2020_hand.EDRPOU_checked]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
5,Вінницька область,місто,Козятинська міська ТГ,"Комунальний заклад ""Навчально-виховний комплек...",I-III,NaN,58,27,4,820,...,21163.339,NaN,02553000000,комунальна,22498,25512362,2020,05,38986337,навчальновиховнийкомплекс3ступлцейшколакозятин...
15,Вінницька область,село,Копайгородська селищна ТГ,Каришківська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,20,16,4,87,...,5847.5,NaN,02554000000,комунальна,6927,02141288,2020,05,13337801,каришквськазагальноосвтняшкола3ступен
19,Вінницька область,село,Барcька міська ТГ,Войнашівська загальноосвітня школа І-ІІІ ступе...,I-III,NaN,45,25,5,410,...,13843,опорна,02505000000,комунальна,6875,02141288,2020,05,13326973,войнашвськазагальноосвтняшкола3ступен
27,Вінницька область,місто,Бершадська міська ТГ,Бершадська загальноосвітня школа І-ІІІ ступені...,I-III,NaN,54,30,9,608,...,17284.594,NaN,02548000000,комунальна,6152,02141294,2020,05,23056598,бершадськазагальноосвтняшкола3ступ3
34,Вінницька область,село,Бершадська міська ТГ,Флоринська загальноосвітня школа І -ІІІ ст.,I-III,NaN,30,19,2,373,...,9907.135,NaN,02548000000,комунальна,6144,02141294,2020,05,26228255,флоринськазагальноосвтняшкола3сту
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9038,м.Київ,місто,місто Київ,Заклад загальної середньої освіти №223 м.Києва,I-III,NaN,48,25,10,946,...,16224.7,NaN,26000000000,комунальна,23168,37498536,2020,80,22878571,223мкив
9096,м.Київ,місто,місто Київ,спеціалізована школа № 317 Святошинського райо...,I-III,NaN,64,30,11,1175,...,20529.9,NaN,26000000000,комунальна,23178,37498536,2020,80,25904889,спецалзованашкола317мкив
9159,м.Київ,місто,місто Київ,"м.Київ, середня загальноосвітня школа № 162, С...",I-III,NaN,60,35,8,1072,...,19150.6,NaN,26000000000,комунальна,23157,37498536,2020,80,22878467,мкивсереднязагальноосвтняшкола162
9183,м.Київ,місто,місто Київ,"Гімназія ""Академія""",I-III,NaN,68,40,8,1217,...,21184.7,NaN,26000000000,комунальна,23179,37498536,2020,80,25805704,гмназяакадем


In [132]:
finance_2020 = dataset_2020_hand.drop(columns=['EDRPOU'], axis=1)
finance_2020 = finance_2020.rename(columns = {'EDRPOU_checked':'EDRPOU'})

In [133]:
finance_2020.to_csv('./fin_data/finance_2020_clean.csv', index = False)

# Data cleaning 2019

In [134]:
datasets[2019].regname = datasets[2019].regname.apply(lambda x: 'м.Київ' if x == 'місто Київ' or x == 'м.Київ' else x+' область')
datasets[2019].regname.unique()

array(['Вінницька область', 'Волинська область',
       'Дніпропетровська область', 'Донецька область',
       'Житомирська область', 'Закарпатська область',
       'Запорізька область', 'Івано-Франківська область',
       'Київська область', 'Кіровоградська область', 'Луганська область',
       'Львівська область', 'Миколаївська область', 'Одеська область',
       'Полтавська область', 'Рівненська область', 'Сумська область',
       'Тернопільська область', 'Харківська область',
       'Херсонська область', 'Хмельницька область', 'Черкаська область',
       'Чернівецька область', 'Чернігівська область', 'м.Київ'],
      dtype=object)

In [135]:
dataset_2019 = datasets[2019].merge(loc[['regname', 'KOATUU_2020_reg']], on=['regname'], how='left')
dataset_2019.head()

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,20,9915.2,11123,ні,2202100000,комунальна,8173,26244113,2019,05
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,30,13692.2,13758.8,ні,2202100000,комунальна,8174,26244107,2019,05
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,36,16184.7,17254.2,ні,2202100000,комунальна,8175,26244136,2019,05
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,32,14321.9,14598.6,ні,2202100000,комунальна,8176,26244099,2019,05
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,33,15061,16011.5,ні,2202100000,комунальна,8177,23064971,2019,05


In [136]:
set(dataset_2019.eolevel.unique())

{' I-III ', 'I', 'I-II', 'I-III', 'II-III', 'III'}

In [137]:
values_to_drop = ['I', 'I-II']

dataset_2019  = dataset_2019[~dataset_2019.eolevel.isin(values_to_drop)]
dataset_2019.reset_index(drop=True, inplace=True)
dataset_2019 

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,20,9915.2,11123,ні,2202100000,комунальна,8173,26244113,2019,05
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,30,13692.2,13758.8,ні,2202100000,комунальна,8174,26244107,2019,05
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,36,16184.7,17254.2,ні,2202100000,комунальна,8175,26244136,2019,05
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,32,14321.9,14598.6,ні,2202100000,комунальна,8176,26244099,2019,05
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,33,15061,16011.5,ні,2202100000,комунальна,8177,23064971,2019,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9661,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад II-III ступ...,II-III,NaN,32,22,8,313,...,11,8137.5,9064.2,ні,26000000000,комунальна,23349,23388780,2019,80
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,12,6033.8,6730.7,ні,26000000000,комунальна,23365,25198658,2019,80
9663,м.Київ,місто,місто Київ,Київська інженерна гімназія,II-III,NaN,110.92,62.25,10,1171,...,39,28151.1,31611.874,ні,26000000000,комунальна,23517,38138802,2019,80
9664,м.Київ,місто,місто Київ,"Гімназія ""Київська Русь"" Дарницького району м....",II-III,NaN,90.61,60,3,890,...,29,21209.9,25467.93,ні,26000000000,комунальна,23518,39143296,2019,80


In [138]:
# check whether there are duplicates by name and location
dataset_2019[dataset_2019.duplicated(subset=['eoname', 'KOATUU_2020_reg'], keep=False)]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
2042,Житомирська область,село,Черняхівський район,Андріївська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,18,11,2,139,...,11,3583.6,3605.4,ні,6322200000,комунальна,16056,6671422,2019,18
2218,Житомирська область,село,Соколівська сільська ОТГ,Андріївська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,17,9,2,97,...,11,3310.9,3704.1,ні,6541000000,комунальна,16035,22056797,2019,18
6777,Тернопільська область,село,Бучацький район,Озерянська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,23,15,2,134,...,11,4325.1,4342.8,ні,19303200000,комунальна,17857,24623237,2019,61
6919,Тернопільська область,село,Озерянська сільська ОТГ,Озерянська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,29,14,1,190,...,11,5711.1,5657.4,ні,19519000000,комунальна,17807,24620894,2019,61


In [139]:
dataset_2019['EDRPOU_checked'] = ''

In [140]:
dataset_2019.loc[dataset_2019.EDRPOU == '06671422', 'EDRPOU_checked'] = '06671422'
dataset_2019.loc[dataset_2019.EDRPOU == '22056797', 'EDRPOU_checked'] = '22056797'
dataset_2019.loc[dataset_2019.EDRPOU == '24623237', 'EDRPOU_checked'] = '24623237'
dataset_2019.loc[dataset_2019.EDRPOU == '24620894', 'EDRPOU_checked'] = '24620894'

In [143]:
dataset_2019[dataset_2019.EDRPOU.str.strip().str.len() != 8].EDRPOU.unique()

array([], dtype=object)

In [142]:
dataset_2019.EDRPOU = dataset_2019.EDRPOU.apply(lambda x: x.zfill(8) if isinstance(x, str) and len(x) >= 6 else None)

In [144]:
dataset_2019[(dataset_2019.EDRPOU.isna())].eoname.unique()

array([], dtype=object)

In [145]:
create_id(dataset_2019, 'eoname', 'id')

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,11123,ні,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,13758.8,ні,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,17254.2,ні,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,14598.6,ні,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,16011.5,ні,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9661,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад II-III ступ...,II-III,NaN,32,22,8,313,...,9064.2,ні,26000000000,комунальна,23349,23388780,2019,80,,загальноосвтнйнавчальнийзаклад3ступфнансовийлц...
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,6730.7,ні,26000000000,комунальна,23365,25198658,2019,80,,загальноосвтнйнавчальнийзакладшколаекстерн
9663,м.Київ,місто,місто Київ,Київська інженерна гімназія,II-III,NaN,110.92,62.25,10,1171,...,31611.874,ні,26000000000,комунальна,23517,38138802,2019,80,,кивськанженернагмназ
9664,м.Київ,місто,місто Київ,"Гімназія ""Київська Русь"" Дарницького району м....",II-III,NaN,90.61,60,3,890,...,25467.93,ні,26000000000,комунальна,23518,39143296,2019,80,,гмназякивськарусьмкив


## Finance 2020

In [148]:
finance_2020[finance_2020.duplicated(subset=['id', 'KOATUU_2020_reg', 'DISO'], keep=False)]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opex,opexplan,hub,budgetcode,ownership,DISO,year,KOATUU_2020_reg,EDRPOU,id


In [151]:
dataset_2019_fin= dataset_2019.merge(finance_2020[['id', 'KOATUU_2020_reg', 'DISO', 'EDRPOU']], on=['id', 'DISO', 'KOATUU_2020_reg'], how='left')
dataset_2019_fin

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,ні,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...,NaN
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,ні,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,ні,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...,NaN
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,ні,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...,NaN
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,ні,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9661,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад II-III ступ...,II-III,NaN,32,22,8,313,...,ні,26000000000,комунальна,23349,23388780,2019,80,,загальноосвтнйнавчальнийзаклад3ступфнансовийлц...,23388780
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,ні,26000000000,комунальна,23365,25198658,2019,80,,загальноосвтнйнавчальнийзакладшколаекстерн,NaN
9663,м.Київ,місто,місто Київ,Київська інженерна гімназія,II-III,NaN,110.92,62.25,10,1171,...,ні,26000000000,комунальна,23517,38138802,2019,80,,кивськанженернагмназ,38138802
9664,м.Київ,місто,місто Київ,"Гімназія ""Київська Русь"" Дарницького району м....",II-III,NaN,90.61,60,3,890,...,ні,26000000000,комунальна,23518,39143296,2019,80,,гмназякивськарусьмкив,39143296


In [152]:
dataset_2019_fin[dataset_2019_fin.EDRPOU_y.notna()]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
6,Вінницька область,місто,місто Козятин,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,32,15,11,312,...,ні,2203100000,комунальна,6936,25844992,2019,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...,25844992
8,Вінницька область,місто,місто Козятин,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,25,19,6,110,...,ні,2203100000,комунальна,7137,26031191,2019,05,,загальноосвтнйнавчальновиховнийкомплекс3ступшк...,25844555
9,Вінницька область,місто,місто Козятин,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,40,17,4,513,...,ні,2203100000,комунальна,7149,25844928,2019,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...,25844928
10,Вінницька область,місто,місто Козятин,"""Загальноосвітня середня школа І-ІІІ ступенів ...",I-III,NaN,39,18,8,450,...,ні,2203100000,комунальна,7191,25844970,2019,05,,загальноосвтнясередняшкола3ступ5козятинськавни...,25844970
11,Вінницька область,місто,місто Козятин,"Комунальний заклад ""Навчально-виховний комплек...",I-III,NaN,63,27,9,781,...,ні,2203100000,комунальна,22498,25512362,2019,05,,навчальновиховнийкомплекс3ступлцейшколакозятин...,38986337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9659,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад І-ІІІ ступе...,I-III,NaN,40,24,8,627,...,ні,26000000000,комунальна,23347,22880941,2019,80,,загальноосвтнйнавчальнийзаклад3ступсереднязага...,22880941
9660,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад I-III ступе...,I-III,NaN,44,28,7,674,...,ні,26000000000,комунальна,23348,22880958,2019,80,,загальноосвтнйнавчальнийзаклад3ступспецалзован...,22880958
9661,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад II-III ступ...,II-III,NaN,32,22,8,313,...,ні,26000000000,комунальна,23349,23388780,2019,80,,загальноосвтнйнавчальнийзаклад3ступфнансовийлц...,23388780
9663,м.Київ,місто,місто Київ,Київська інженерна гімназія,II-III,NaN,110.92,62.25,10,1171,...,ні,26000000000,комунальна,23517,38138802,2019,80,,кивськанженернагмназ,38138802


In [153]:
dataset_2019_fin.loc[(dataset_2019_fin['EDRPOU_y'].notna())&(dataset_2019_fin.EDRPOU_checked == ''), 'EDRPOU_checked'] = dataset_2019_fin.loc[(dataset_2019_fin['EDRPOU_y'].notna())&(dataset_2019_fin.EDRPOU_checked == ''), 'EDRPOU_y']

In [154]:
dataset_2019_fin = dataset_2019_fin.drop(columns=['EDRPOU_y'], axis=1)
dataset_2019_fin = dataset_2019_fin.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [156]:
dataset_2019_fin[dataset_2019_fin['EDRPOU_checked'] == '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,11123,ні,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,13758.8,ні,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,17254.2,ні,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,14598.6,ні,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,16011.5,ні,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9653,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад І-ІІІ ступе...,I-III,NaN,55,20,21,612,...,20146.2,ні,26000000000,комунальна,23341,22880906,2019,80,,загальноосвтнйнавчальнийзаклад3ступспецалзован...
9654,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад І-ІІІ ступе...,I-III,NaN,44,18,15,635,...,16906.9,ні,26000000000,комунальна,23342,22880912,2019,80,,загальноосвтнйнавчальнийзаклад3ступсереднязага...
9655,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад I-III ступе...,I-III,NaN,61,23,13,937,...,22437.6,ні,26000000000,комунальна,23343,22881917,2019,80,,загальноосвтнйнавчальнийзаклад3ступспецалзован...
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,6730.7,ні,26000000000,комунальна,23365,25198658,2019,80,,загальноосвтнйнавчальнийзакладшколаекстерн


## Mon 2023

In [157]:
dataset_2019_mon= dataset_2019_fin.merge(schools_data_id[['id', 'KOATUU_2020_reg', 'EDRPOU']], on=['id', 'KOATUU_2020_reg'], how='left')
dataset_2019_mon

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,ні,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...,NaN
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,ні,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,ні,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...,NaN
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,ні,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...,NaN
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,ні,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9661,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад II-III ступ...,II-III,NaN,32,22,8,313,...,ні,26000000000,комунальна,23349,23388780,2019,80,23388780,загальноосвтнйнавчальнийзаклад3ступфнансовийлц...,23388780
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,ні,26000000000,комунальна,23365,25198658,2019,80,,загальноосвтнйнавчальнийзакладшколаекстерн,NaN
9663,м.Київ,місто,місто Київ,Київська інженерна гімназія,II-III,NaN,110.92,62.25,10,1171,...,ні,26000000000,комунальна,23517,38138802,2019,80,38138802,кивськанженернагмназ,38138802
9664,м.Київ,місто,місто Київ,"Гімназія ""Київська Русь"" Дарницького району м....",II-III,NaN,90.61,60,3,890,...,ні,26000000000,комунальна,23518,39143296,2019,80,39143296,гмназякивськарусьмкив,39143296


In [158]:
dataset_2019_mon.loc[(dataset_2019_mon['EDRPOU_y'].notna())&(dataset_2019_mon.EDRPOU_checked == ''), 'EDRPOU_checked'] = dataset_2019_mon.loc[(dataset_2019_mon['EDRPOU_y'].notna())&(dataset_2019_mon.EDRPOU_checked == ''), 'EDRPOU_y']

In [159]:
dataset_2019_mon = dataset_2019_mon.drop(columns=['EDRPOU_y'], axis=1)
dataset_2019_mon = dataset_2019_mon.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [161]:
dataset_2019_mon[dataset_2019_mon['EDRPOU_checked'] == '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,11123,ні,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,13758.8,ні,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,17254.2,ні,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,14598.6,ні,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,16011.5,ні,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9623,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів №135 з пог...,I-III,NaN,55,18,9,567,...,16380.497,ні,26000000000,комунальна,23288,26125779,2019,80,,спецалзованашкола3ступ135зпоглибленимвивченяма...
9624,м.Київ,місто,місто Київ,спеціалізована школа №138 І-ІІІ ступенів з вив...,I-III,NaN,51,21,10,366,...,14490.684,ні,26000000000,комунальна,23289,22881875,2019,80,,спецалзованашкола1383ступзвивченямпредметвприр...
9654,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад І-ІІІ ступе...,I-III,NaN,44,18,15,635,...,16906.9,ні,26000000000,комунальна,23342,22880912,2019,80,,загальноосвтнйнавчальнийзаклад3ступсереднязага...
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,6730.7,ні,26000000000,комунальна,23365,25198658,2019,80,,загальноосвтнйнавчальнийзакладшколаекстерн


### short name

In [162]:
dataset_2019_mon_short= dataset_2019_mon.merge(schools_data_id_short[['id', 'KOATUU_2020_reg', 'EDRPOU']], on=['id', 'KOATUU_2020_reg'], how='left')
dataset_2019_mon_short

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,ні,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...,NaN
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,ні,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,ні,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...,NaN
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,ні,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...,NaN
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,ні,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9661,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад II-III ступ...,II-III,NaN,32,22,8,313,...,ні,26000000000,комунальна,23349,23388780,2019,80,23388780,загальноосвтнйнавчальнийзаклад3ступфнансовийлц...,NaN
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,ні,26000000000,комунальна,23365,25198658,2019,80,,загальноосвтнйнавчальнийзакладшколаекстерн,NaN
9663,м.Київ,місто,місто Київ,Київська інженерна гімназія,II-III,NaN,110.92,62.25,10,1171,...,ні,26000000000,комунальна,23517,38138802,2019,80,38138802,кивськанженернагмназ,38138802
9664,м.Київ,місто,місто Київ,"Гімназія ""Київська Русь"" Дарницького району м....",II-III,NaN,90.61,60,3,890,...,ні,26000000000,комунальна,23518,39143296,2019,80,39143296,гмназякивськарусьмкив,NaN


In [163]:
dataset_2019_mon_short.loc[(dataset_2019_mon_short['EDRPOU_y'].notna())&(dataset_2019_mon_short.EDRPOU_checked == ''), 'EDRPOU_checked'] = dataset_2019_mon_short.loc[(dataset_2019_mon_short['EDRPOU_y'].notna())&(dataset_2019_mon_short.EDRPOU_checked == ''), 'EDRPOU_y']

In [ ]:
dataset_2019_mon_short = dataset_2019_mon_short.drop(columns=['EDRPOU_y'], axis=1)
dataset_2019_mon_short = dataset_2019_mon_short.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [165]:
dataset_2019_mon_short[dataset_2019_mon_short['EDRPOU_checked'] == '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,ні,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...,NaN
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,ні,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,ні,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...,NaN
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,ні,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...,NaN
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,ні,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9623,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів №135 з пог...,I-III,NaN,55,18,9,567,...,ні,26000000000,комунальна,23288,26125779,2019,80,,спецалзованашкола3ступ135зпоглибленимвивченяма...,NaN
9624,м.Київ,місто,місто Київ,спеціалізована школа №138 І-ІІІ ступенів з вив...,I-III,NaN,51,21,10,366,...,ні,26000000000,комунальна,23289,22881875,2019,80,,спецалзованашкола1383ступзвивченямпредметвприр...,NaN
9654,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад І-ІІІ ступе...,I-III,NaN,44,18,15,635,...,ні,26000000000,комунальна,23342,22880912,2019,80,,загальноосвтнйнавчальнийзаклад3ступсереднязага...,NaN
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,ні,26000000000,комунальна,23365,25198658,2019,80,,загальноосвтнйнавчальнийзакладшколаекстерн,NaN


## youcontrol

In [166]:
dataset_2019_youcontrol_long= dataset_2019_mon_short.merge(youcontrol_long[['id', 'EDRPOU']], on=['id'], how='left')
dataset_2019_youcontrol_long

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y,EDRPOU
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...,NaN,NaN
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN,NaN
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...,NaN,NaN
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...,NaN,NaN
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9661,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад II-III ступ...,II-III,NaN,32,22,8,313,...,26000000000,комунальна,23349,23388780,2019,80,23388780,загальноосвтнйнавчальнийзаклад3ступфнансовийлц...,NaN,23388780
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,26000000000,комунальна,23365,25198658,2019,80,,загальноосвтнйнавчальнийзакладшколаекстерн,NaN,25198658
9663,м.Київ,місто,місто Київ,Київська інженерна гімназія,II-III,NaN,110.92,62.25,10,1171,...,26000000000,комунальна,23517,38138802,2019,80,38138802,кивськанженернагмназ,38138802,38138802
9664,м.Київ,місто,місто Київ,"Гімназія ""Київська Русь"" Дарницького району м....",II-III,NaN,90.61,60,3,890,...,26000000000,комунальна,23518,39143296,2019,80,39143296,гмназякивськарусьмкив,NaN,39143296


In [167]:
dataset_2019_youcontrol_long.loc[(dataset_2019_youcontrol_long['EDRPOU_y'].notna())&(dataset_2019_youcontrol_long.EDRPOU_checked == ''), 'EDRPOU_checked'] = dataset_2019_youcontrol_long.loc[(dataset_2019_youcontrol_long['EDRPOU_y'].notna())&(dataset_2019_youcontrol_long.EDRPOU_checked == ''), 'EDRPOU_y']

In [168]:
dataset_2019_youcontrol_long = dataset_2019_youcontrol_long.drop(columns=['EDRPOU_y'], axis=1)
dataset_2019_youcontrol_long = dataset_2019_youcontrol_long.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [170]:
dataset_2019_youcontrol_long[dataset_2019_youcontrol_long['EDRPOU_checked'] == '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,ні,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...,NaN
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,ні,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,ні,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...,NaN
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,ні,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...,NaN
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,ні,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9623,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів №135 з пог...,I-III,NaN,55,18,9,567,...,ні,26000000000,комунальна,23288,26125779,2019,80,,спецалзованашкола3ступ135зпоглибленимвивченяма...,NaN
9624,м.Київ,місто,місто Київ,спеціалізована школа №138 І-ІІІ ступенів з вив...,I-III,NaN,51,21,10,366,...,ні,26000000000,комунальна,23289,22881875,2019,80,,спецалзованашкола1383ступзвивченямпредметвприр...,NaN
9654,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад І-ІІІ ступе...,I-III,NaN,44,18,15,635,...,ні,26000000000,комунальна,23342,22880912,2019,80,,загальноосвтнйнавчальнийзаклад3ступсереднязага...,NaN
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,ні,26000000000,комунальна,23365,25198658,2019,80,,загальноосвтнйнавчальнийзакладшколаекстерн,25198658


In [171]:
dataset_2019_youcontrol_short= dataset_2019_youcontrol_long.merge(youcontrol_short[['id', 'EDRPOU']], on=['id'], how='left')
dataset_2019_youcontrol_short

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_x,EDRPOU_y
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...,NaN,NaN
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN,NaN
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...,NaN,NaN
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...,NaN,NaN
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9661,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад II-III ступ...,II-III,NaN,32,22,8,313,...,26000000000,комунальна,23349,23388780,2019,80,23388780,загальноосвтнйнавчальнийзаклад3ступфнансовийлц...,23388780,NaN
9662,м.Київ,місто,місто Київ,"Загальноосвітній навчальний заклад ""Школа екст...",I-III,NaN,35,10,8,326,...,26000000000,комунальна,23365,25198658,2019,80,,загальноосвтнйнавчальнийзакладшколаекстерн,25198658,25198658
9663,м.Київ,місто,місто Київ,Київська інженерна гімназія,II-III,NaN,110.92,62.25,10,1171,...,26000000000,комунальна,23517,38138802,2019,80,38138802,кивськанженернагмназ,38138802,NaN
9664,м.Київ,місто,місто Київ,"Гімназія ""Київська Русь"" Дарницького району м....",II-III,NaN,90.61,60,3,890,...,26000000000,комунальна,23518,39143296,2019,80,39143296,гмназякивськарусьмкив,39143296,NaN


In [172]:
dataset_2019_youcontrol_short.loc[(dataset_2019_youcontrol_short['EDRPOU_y'].notna())&(dataset_2019_youcontrol_short.EDRPOU_checked == ''), 'EDRPOU_checked'] = dataset_2019_youcontrol_short.loc[(dataset_2019_youcontrol_short['EDRPOU_y'].notna())&(dataset_2019_youcontrol_short.EDRPOU_checked == ''), 'EDRPOU_y']

In [173]:
dataset_2019_youcontrol_short = dataset_2019_youcontrol_short.drop(columns=['EDRPOU_y'], axis=1)
dataset_2019_youcontrol_short = dataset_2019_youcontrol_short.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [175]:
dataset_2019_youcontrol_short[dataset_2019_youcontrol_short['EDRPOU_checked'] == '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,ні,2202100000,комунальна,8173,26244113,2019,05,,загальноосвтняшкола3ступ1комунальнавласностмжм...,NaN
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,ні,2202100000,комунальна,8174,26244107,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,ні,2202100000,комунальна,8175,26244136,2019,05,,загальноосвтняшкола3ступ3комунальнавласностмжм...,NaN
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,ні,2202100000,комунальна,8176,26244099,2019,05,,загальноосвтняшкола3ступ4комунальнавластностте...,NaN
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,ні,2202100000,комунальна,8177,23064971,2019,05,,навчальновиховнийкомплексзагальноосвтняшкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9617,м.Київ,місто,місто Київ,Київська спеціалізована школа І-ІІІ ступенів №...,I-III,NaN,44,19,5,467,...,ні,26000000000,комунальна,23282,26187705,2019,80,,кивськаспецалзованашкола3ступ91зпоглибленимвив...,NaN
9623,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів №135 з пог...,I-III,NaN,55,18,9,567,...,ні,26000000000,комунальна,23288,26125779,2019,80,,спецалзованашкола3ступ135зпоглибленимвивченяма...,NaN
9624,м.Київ,місто,місто Київ,спеціалізована школа №138 І-ІІІ ступенів з вив...,I-III,NaN,51,21,10,366,...,ні,26000000000,комунальна,23289,22881875,2019,80,,спецалзованашкола1383ступзвивченямпредметвприр...,NaN
9654,м.Київ,місто,місто Київ,Загальноосвітній навчальний заклад І-ІІІ ступе...,I-III,NaN,44,18,15,635,...,ні,26000000000,комунальна,23342,22880912,2019,80,,загальноосвтнйнавчальнийзаклад3ступсереднязага...,NaN


In [178]:
dataset_2019_youcontrol_short[dataset_2019_youcontrol_short['EDRPOU_checked'] == ''][['eoname', 'EDRPOU']].to_csv('./finance_2019_for_bot.csv', index = False)

In [ ]:
# datasets[2019].loc[datasets[2019]['eoname'] == 'Марянівський навчально-виховний комплекс "ДНЗ-ЗОШ" Іступеня', 'EDRPOU'] = '26224046'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Лобойківський заклад загальної середньої освіти Петриківської районної ради Дніпропетровської області', 'EDRPOU'] = '26460466'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Середня загальноосвітня школа І-ІІ ступенів села Вартиківці Кельменецького району Чернівецької області', 'EDRPOU'] = '31967606'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Рипенська загальноосвітня школа І ступеня Путильської районної ради Путильський район Чернівецька область', 'EDRPOU'] = '25811521'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Рижівська загальноосвітня школа І ступеня села Рижа Путильського району Чернівецької області', 'EDRPOU'] = '25811484'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Великобушинська загальноосвітня школа І-ІІ ступенів Немирівської районної ради Вінницької області', 'EDRPOU'] = '25513410'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Мирненська загальноосвітня школа І ступеня Надвірнянської районної ради Івано-Франківської області', 'EDRPOU'] = '20567542'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Завадівський навчально-виховний комплекс І-ІІ ступенів Турківської районної ради Львівської обасті', 'EDRPOU'] = '22417999'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Херсонський навчально-виховний комплекс "Загальноосвітня школа ІІ ступеня-ліцей журналістики, економіки та правознавства" Херсонської міської ради', 'EDRPOU'] = '24114961'

In [ ]:
# datasets[2019][datasets[2019].duplicated(subset=['EDRPOU'], keep=False)]

In [ ]:
# datasets[2020].loc[datasets[2020]['EDRPOU'].apply(len) == 7, 'EDRPOU'] = '0' + datasets[2020].loc[datasets[2020]['EDRPOU'].apply(len) == 7, 'EDRPOU']
# datasets[2020].loc[datasets[2020]['eoname'] == 'Турбівська загальноосвітня школа І-ІІІ ст. №2 _x000D_\nЛиповецької районної ради _x000D_\nВінницької області_x000D_\n_x000D_\n', 'EDRPOU'] = '21723820'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Середня загальноосвітня школа І - ІІІ ступенів села Уладівка Літинський район Вінницька область', 'EDRPOU'] = '26229088'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Середня загальноосвітня школа І-ІІІ ступенів с. Івча', 'EDRPOU'] = '26223660'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Комунальний заклад "Загальноосвітня школа І-ІІІ ступенів № 19 Вінницької міської ради"', 'EDRPOU'] = '26235447'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Тисаашванська загальноосвітня школа І-ІІ ступенів Ужгородської районної ради Закарпатської області', 'EDRPOU'] = '22103539'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Пшеничниківський навчально-виховний комплекс "Загальноосвітня школа І-ІІ ступенів - дошкільний навчальний заклад" Тисменицької районної ради Івано-Франківської області', 'EDRPOU'] = '20563076'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Старобільська загальноосвітня школа ІІ-ІІІ ступенів №4 Старобільської районної ради Луганської області', 'EDRPOU'] = '33199525'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Заклад загальноосвітньої середньої освіти І ступення села Смеречка Старосамбірського району Львівської області', 'EDRPOU'] = '00000000'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Загальноосвітня середня школа І ступеня села Дністрик Старосамбірського району Львівської області', 'EDRPOU'] = '00000000'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Заклад загальної середньої освіти І ступеня села Головецько-Золотнівець Старосамбірського району Львівської області', 'EDRPOU'] = '00000000'
# datasets[2020].loc[datasets[2020]['eoname'] == "Воле - Блажівська середня загальноосвітня школа І ступеня Чукв'янської сільської ради Самбірського району Львівської області", 'EDRPOU'] = '22378609'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Загальноосвітня школа І ступеня села Виноградне', 'EDRPOU'] = '24625450'
# datasets[2020].loc[datasets[2020]['eoname'] == ' Херсонський навчально-виховний комплекс "Загальноосвітня школа ІІ ст-ліцей журналістики, економіки та правознавства" Херсонської міської ради', 'EDRPOU'] = '24114961'

In [ ]:
# concatenated_df = pd.concat(list(datasets.values()), ignore_index=True)
# concatenated_df

In [ ]:
# concatenated_df[concatenated_df['Код ЄДРПОУ'].notna()][concatenated_df[concatenated_df['Код ЄДРПОУ'].notna()]['Код ЄДРПОУ'].apply(len) != 8]

In [ ]:
# concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']] = concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']].fillna(-10000)
# concatenated_df = concatenated_df.astype({'teachstuff': 'int64', 'nonteachstuff': 'int64', 'teachstuffretage': 'int64', 'pupilswithpreschool': 'int64', 'budgetcode': 'int64', 'DISO': 'int64', 'Код ЄДРПОУ': 'int64'})
# concatenated_df = concatenated_df.astype({'pupilswithpreschool': 'str', 'budgetcode': 'str', 'DISO': 'str', 'Код ЄДРПОУ': 'str'})
# concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']] = concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']].replace('-10000', np.NaN)
# concatenated_df

In [ ]:
# concatenated_df.to_csv('Scools_stats.csv', index=False, encoding='utf-8')